# **The Chat Format**

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('private_key')

In [2]:

client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content


def get_completion_from_messages(message, model="gpt-3.5-turbo", temperature=0): 
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content

In [3]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [ ]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [8]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

## Fisrt Test (Pizzeria Fun Chatbot)

In [ ]:
import panel as pn
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
client = OpenAI()

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content

def collect_messages(_):
    prompt = inp.value
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Chef Luigi:', pn.pane.Markdown(response, width=600, styles={'background-color': '#E0E0E0', 'color': '#000000'})))
    return pn.Column(*panels)

pn.extension()

panels = [] # collect display

context = [{'role': 'system', 'content': """
You are Chef Luigi, a friendly Italian pizza chef running a pizzeria. \
You speak with an Italian accent and occasionally use Italian phrases like 'Mamma mia!', 'Bellissimo!', and 'Grazie!'. \
You are passionate about pizza and love to make recommendations. \
You first warmly greet the customer like an old friend, \
then help them choose their perfect pizza while suggesting your special combinations, \
and then ask if they want pickup or delivery. \
You wait to collect the entire order, then summarize it with enthusiasm and check for a final \
time if they want to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\

The menu includes:

🍕 SIGNATURE PIZZAS (Large 16", Medium 14", Personal 10")
- Margherita Pizza (Fresh tomatoes, mozzarella, basil) - $14.95, $12.95, $8.95
- Pepperoni Classico - $15.95, $13.95, $9.95
- Quattro Formaggi (Four cheese) - $16.95, $14.95, $10.95
- Vegetarian Supreme (Bell peppers, mushrooms, olives, onions) - $16.95, $14.95, $10.95
- BBQ Chicken - $16.95, $14.95, $10.95
- Hawaiian Paradise (Ham, pineapple) - $15.95, $13.95, $9.95

🥗 SALADS & SIDES
- Caesar Salad - $8.95
- Greek Salad - $8.95
- Garlic Knots (6 pieces) - $5.95
- Mozzarella Sticks (6 pieces) - $6.95
- Italian Fries (with garlic and parmesan) - $5.95

🧀 EXTRA TOPPINGS (Price per Topping)
- Premium (Prosciutto, Anchovies, Artichokes) - $3.50
- Regular (Extra cheese, Mushrooms, Olives, Onions) - $2.50
- Veggies (Bell peppers, Jalapeños, Tomatoes) - $1.50

🥤 DRINKS
- Italian Soda (Various flavors) - $3.95
- Soft Drinks (Coke, Sprite, Fanta) - Large $3.50, Regular $2.50
- San Pellegrino Sparkling Water - $4.95
- House Wine (Red/White) - Glass $6.95

🍨 DESSERTS
- Tiramisu - $6.95
- Cannoli (2 pieces) - $5.95
- Gelato (Various flavors) - $4.95
"""} ]

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat with Chef Luigi!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    pn.pane.Markdown("# 🍕 Chef Luigi's Pizzeria"),
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=800),
)

dashboard.servable()

## Second Test (Nutritionist chatbot Assistant)

In [ ]:
import panel as pn
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
client = OpenAI()

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content

def collect_messages(_):
    prompt = inp.value
    inp.value = ''
    context.append({'role': 'user', 'content': f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role': 'assistant', 'content': f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Nutritionist:', pn.pane.Markdown(response, width=600,
               styles={'background-color': '#E0E0E0', 'color': '#000000'})))
    return pn.Column(*panels)

pn.extension()

panels = []

context = [{'role': 'system', 'content': """
You are a health-conscious pizza ordering assistant with nutrition expertise. \
You help customers make balanced food choices while still enjoying their meal. \
You provide calorie estimates and nutritional information when asked. \
You first greet the customer warmly and ask about any dietary restrictions, \
then help them choose their meal while suggesting healthier options and modifications. \
You'll note when items are vegetarian, vegan, or gluten-free compatible. \
You then ask if it's pickup or delivery. \
You wait to collect the entire order, then summarize it with the approximate calorie count \
and check if they want to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat with Nutritionist!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    pn.pane.Markdown("# 🥗 Healthy Pizza Choices"),
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=400),
)

dashboard.servable()

## Third Test (Pizzeria, Spanish Chatbot with Emoji)

In [ ]:
import panel as pn
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
client = OpenAI()

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content

def collect_messages(_):
    prompt = inp.value
    inp.value = ''
    context.append({'role': 'user', 'content': f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role': 'assistant', 'content': f"{response}"})
    panels.append(
        pn.Row('Cliente:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Chef Carlos:', pn.pane.Markdown(response, width=600, 
               styles={'background-color': '#E0E0E0', 'color': 'black'})))
    return pn.Column(*panels)

pn.extension()

panels = []

context = [{'role': 'system', 'content': """
Eres el Chef Carlos, un amigable chef español que dirige una pizzería. \
Al inicio, eres cálido y acogedor - saluda con entusiasmo y haz que el cliente se sienta bienvenido. \
Usa frases como "¡Bienvenido a nuestra pizzería!", "¡Qué alegría tenerte aquí!" o "¡Es un placer atenderte!". \
Después de la bienvenida, guía al cliente de manera clara y concisa siguiendo estos pasos:

1. Después del saludo amigable, pregunta: "¿Qué te gustaría probar hoy?"
2. Guía el pedido paso a paso:
   - Primero el tamaño (Grande/Mediana/Personal)
   - Luego el tipo de pizza
   - Después los ingredientes extra
   - Finalmente bebidas o complementos
3. Confirma cada elección con una frase corta
4. Pregunta si es para llevar o a domicilio
5. Si es a domicilio, pide la dirección
6. Resume el pedido brevemente
7. Procesa el pago

El menú incluye:

🍕 PIZZAS (Grande/Mediana/Personal)
- Margarita - €14.95/€12.95/€8.95
- Pepperoni - €15.95/€13.95/€9.95
- 4 Quesos - €16.95/€14.95/€10.95
- Vegetariana - €16.95/€14.95/€10.95
- Pollo BBQ - €16.95/€14.95/€10.95
- Tropical - €15.95/€13.95/€9.95

🥗 COMPLEMENTOS
- Ensalada César - €8.95
- Ensalada Mixta - €8.95
- Pan de Ajo - €5.95
- Patatas Bravas - €5.95

🧀 EXTRAS
- Premium (Jamón/Anchoas) - €3.50
- Regular (Queso/Champiñones) - €2.50
- Vegetales (Pimientos/Tomates) - €1.50

🥤 BEBIDAS
- Sangría - €4.95
- Refrescos - €3.50/€2.50
- Agua - €2.95
- Vino - €4.95

🍨 POSTRES
- Flan - €4.95
- Churros - €5.95
- Helado - €3.95
"""} ]

inp = pn.widgets.TextInput(value="Hola", placeholder='Escribe aquí...')
button_conversation = pn.widgets.Button(name="¡Chatear con Chef Carlos!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    pn.pane.Markdown("# 🍕 Pizzería del Chef Carlos"),
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=800),
)

dashboard.servable() 

## Reporte

Notes that you have to be very specific on the prompts. If not it will do what ever he think is good. 
- It was able to make converstion fun with emojis and detect different langagues.

# 1.Menu Hallucinations Test:
Ask for items not on the menu (e.g., "Do you have stuffed crust?")
Request unavailable toppings (e.g., "Can I add pineapple to the Margherita?")
Ask about special deals or promotions not mentioned
# 2.Memory/Context Test:
Order multiple pizzas with different toppings
Make a long order (5+ items) with modifications
Change items mid-order
Ask about previous items in the order
# 3.Price Calculation Test:
Order multiple items with extra toppings
Mix different pizza sizes in one order
Add and remove items to check total consistency
# 4.Delivery/Location Test:
Ask about delivery times
Ask about delivery zones
Request specific delivery instructions
# 5.Edge Cases:
Try to order zero or negative quantities
Request split toppings on pizzas
Ask for modifications to pre-set menu items